In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load example parquet file
df = pd.read_parquet('../data/asl-signs/train_landmark_files/16069/100015657.parquet')
df.shape

(57015, 7)

In [4]:
df.head()

,frame,row_id,type,landmark_index,x,y,z
0,103,103-face-0,face,0,0.437886,0.437599,-0.051134
1,103,103-face-1,face,1,0.443258,0.392901,-0.067054
2,103,103-face-2,face,2,0.443997,0.409998,-0.042990
3,103,103-face-3,face,3,0.435256,0.362771,-0.039492
4,103,103-face-4,face,4,0.443780,0.381762,-0.068013


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57015 entries, 0 to 57014
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   frame           57015 non-null  int16  
 1   row_id          57015 non-null  object 
 2   type            57015 non-null  object 
 3   landmark_index  57015 non-null  int16  
 4   x               53193 non-null  float64
 5   y               53193 non-null  float64
 6   z               53193 non-null  float64
dtypes: float64(3), int16(2), object(2)
memory usage: 2.4+ MB


In [5]:
df.describe() # doesn't make much sense

,frame,landmark_index,x,y,z
count,57015.00000,57015.000000,53193.000000,53193.000000,53193.000000
mean,155.00000,202.994475,0.438283,0.457483,-0.047699
std,30.30978,146.791506,0.115117,0.268900,0.294060
min,103.00000,0.000000,-0.042923,0.248094,-2.838325
25%,129.00000,60.000000,0.372952,0.361456,-0.039124
50%,155.00000,196.000000,0.424128,0.409588,-0.009181
75%,181.00000,332.000000,0.489478,0.474334,0.025908
max,207.00000,467.000000,1.197836,2.591290,1.587503
